In [1]:
import numpy as np

In [2]:
from nndl.EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K

In [38]:
import pandas as pd
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten,Dropout
from tensorflow.keras.layers import Conv2D,BatchNormalization,MaxPooling2D,Reshape
from tensorflow.keras.utils import to_categorical


In [8]:
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.utils.viz import plot_confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

In [9]:
from matplotlib import pyplot as plt

In [16]:
from utils.preprocess import getData

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
X_train, Y_train, X_validate, Y_validate, X_test, y_test = getData()

### Shape of data

In [18]:
print('Training data shape: {}'.format(X_train.shape))
print('Valid data shape: {}'.format(X_validate.shape))
print('Test data shape: {}'.format(X_test.shape))

print('Training target shape: {}'.format(Y_train.shape))
print('Valid target shape: {}'.format(Y_validate.shape))
print('Test target shape: {}'.format(y_test.shape))

Training data shape: (1615, 1000, 1, 22)
Valid data shape: (500, 1000, 1, 22)
Test data shape: (443, 1000, 1, 22)
Training target shape: (1615, 4)
Valid target shape: (500, 4)
Test target shape: (443, 4)


In [19]:
K.set_image_data_format('channels_last')

In [20]:
# convert labels to one-hot encodings.
Y_train      = np_utils.to_categorical(Y_train-1)
Y_validate      = np_utils.to_categorical(Y_validate-1)
y_test       = np_utils.to_categorical(y_test-1)

In [22]:
kernels, chans, samples = 1, 22, 1000

# convert data to NHWC (trials, channels, samples, kernels) format. Data 
# contains 60 channels and 151 time-points. Set the number of kernels to 1.
X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
X_validate      = X_validate.reshape(X_validate.shape[0], chans, samples, kernels)
X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)

In [23]:
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (1615, 22, 1000, 1)
1615 train samples
443 test samples


In [24]:
# configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
# model configurations may do better, but this is a good starting point)
model = EEGNet(nb_classes = 4, Chans = chans, Samples = samples, 
               dropoutRate = 0.5, kernLength = 32, F1 = 8, D = 2, F2 = 16, 
               dropoutType = 'Dropout')

In [48]:
# compile the model and set the optimizers
"""
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])
"""
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['accuracy'],sample_weight_mode="temporal")

# count number of parameters in the model
numParams    = model.count_params()    

# set a valid path for your system to record model checkpoints
checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                               save_best_only=True)

In [49]:
# the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
# the weights all to be 1

#class_weights = {0:1, 1:1, 2:1, 3:1}
sample_weight = {0:1, 1:1, 2:1, 3:1}


In [50]:
"""
fittedModel = model.fit(X_train, Y_train, batch_size = 16, epochs = 300, 
                        verbose = 2, validation_data=(X_validate, Y_validate),
                        callbacks=[checkpointer], class_weight = class_weights)
"""

fittedModel = model.fit(X_train, Y_train, batch_size = 16, epochs = 300, 
                        verbose = 2, validation_data=(X_validate, Y_validate),
                        callbacks=[checkpointer], sample_weight = sample_weight)

# load optimal weights
model.load_weights('/tmp/checkpoint.h5')

AttributeError: 'int' object has no attribute 'shape'